In [1]:
import keras.applications.densenet as densenet
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
import numpy as np

#이미지 생성용
from keras.preprocessing.image import ImageDataGenerator
from collections import Counter

# 콜백 함수
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

#시간 구하기용
from datetime import datetime

#디렉토리 생성용
import os

Using TensorFlow backend.


In [2]:
# 설정값들
IMAGE_SIZE = 128
IMAGE_DEPTH = 3
LABEL_NUM = 16

# 이미지 volume 값
input_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_DEPTH)

#GetCNN Model
model = densenet.DenseNet121(include_top=True, weights=None, input_shape=input_shape, pooling='max', classes=LABEL_NUM)

# 모델 출력
model.summary()

#Compling CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [3]:
# Validation값에 따라 훈련 조기종료 설정
early_stopping = EarlyStopping(patience=2)

# Class Weights 계산
# 가장 많은 클레스의 수 / 각 클레스의 수
# 클레스 = 각 라벨에 해당하는 데이터 수
'''
counter = Counter(training_set.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
'''

#오늘 날짜_시간
NOW = datetime.now().strftime('%Y%m%d_%H_%M_%S')

#텐서보드 로그
tensorboard = TensorBoard(log_dir="./Logs/DenseNet/" + NOW)

#디렉토리 생성
def MakeDirectory(dir_path):
    try:
        if not(os.path.isdir(dir_path)):
            os.makedirs(os.path.join(dir_path))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

In [4]:
#훈련 6개의 데이터 셋으로 진행(Validation 데이터 셋이 다름)
DATASET_NUM = 6
EPOCH_NUM = 20
BATCH_SIZE = 32
CYCLE_NUM = 8

#훈련용 이미지 생성기
train_datagen = ImageDataGenerator(horizontal_flip = True)
validation_datagen = ImageDataGenerator(horizontal_flip = True)

# Weight, Model 저장용 폴더 생성
model_save_dir = "./Models/DenseNet/" + NOW
MakeDirectory(model_save_dir)

#모델 저장
model_json = model.to_json()
with open(model_save_dir + "/model_densenet_" + NOW + ".json", "w") as json_file : 
    json_file.write(model_json)
             
#몇 사이클 돌지
for cycle in range(0, CYCLE_NUM):
    
    print("Cycle ", (cycle + 1))
    
    # 데이터셋 수 만큼 훈련
    for index in range(1, (DATASET_NUM + 1)):

        print("Training ", index)

        # 데이터 셋 생성
        dir_path = "".join(['./Dataset_Multi/Dataset', str(index)])
        training_set = train_datagen.flow_from_directory(dir_path + '/Train',
                                                    target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                    batch_size = BATCH_SIZE)

        validation_set = validation_datagen.flow_from_directory(dir_path + '/Validation',
                                                   target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                   batch_size = BATCH_SIZE)

        #배치 수 계산
        train_step_epoch = int(len(training_set.classes) / BATCH_SIZE)
        validation_step_epoch = int(len(validation_set.classes) / BATCH_SIZE)
        
        #훈련!
        model.fit_generator(training_set,
                            steps_per_epoch = train_step_epoch,
                            epochs = EPOCH_NUM,
                            validation_data = validation_set,
                            validation_steps = validation_step_epoch,
                            #class_weight=class_weights,
                            shuffle=True,
                            callbacks=[tensorboard, early_stopping])

        print("Traing ", index, " done")
        
        #오늘 날짜_시간
        current_time = datetime.now().strftime('%Y%m%d_%H_%M_%S')
        
        # Weight 저장
        model.save_weights(model_save_dir + "/weight_densenet_" + current_time + ".h5")
        print("Saved model to disk\n")

Cycle  1
Training  1
Found 62573 images belonging to 16 classes.
Found 15875 images belonging to 16 classes.
Epoch 1/20
1955/1955 [==============================] - 869s 445ms/step - loss: 2.4718 - acc: 0.2569 - val_loss: 2.4800 - val_acc: 0.2538
Epoch 2/20
1955/1955 [==============================] - 334s 171ms/step - loss: 2.4172 - acc: 0.2622 - val_loss: 2.6046 - val_acc: 0.2537
Epoch 3/20
1955/1955 [==============================] - 335s 171ms/step - loss: 2.3796 - acc: 0.2676 - val_loss: 2.5501 - val_acc: 0.2410
Traing  1  done
Saved model to disk

Training  2
Found 68806 images belonging to 16 classes.
Found 9642 images belonging to 16 classes.
Epoch 1/20
2150/2150 [==============================] - 832s 387ms/step - loss: 2.3499 - acc: 0.2715 - val_loss: 2.5118 - val_acc: 0.2491
Epoch 2/20
2150/2150 [==============================] - 360s 168ms/step - loss: 2.3147 - acc: 0.2739 - val_loss: 2.3925 - val_acc: 0.2501
Epoch 3/20
2150/2150 [==============================] - 360s 167m

2150/2150 [==============================] - 360s 167ms/step - loss: 0.3754 - acc: 0.8716 - val_loss: 0.7124 - val_acc: 0.7646
Traing  2  done
Saved model to disk

Training  3
Found 62272 images belonging to 16 classes.
Found 16176 images belonging to 16 classes.
Epoch 1/20
1946/1946 [==============================] - 335s 172ms/step - loss: 0.3204 - acc: 0.8940 - val_loss: 0.5416 - val_acc: 0.8013
Epoch 2/20
1946/1946 [==============================] - 335s 172ms/step - loss: 0.2716 - acc: 0.9097 - val_loss: 0.5904 - val_acc: 0.7818
Epoch 3/20
1946/1946 [==============================] - 335s 172ms/step - loss: 0.2594 - acc: 0.9124 - val_loss: 0.6546 - val_acc: 0.7663
Traing  3  done
Saved model to disk

Training  4
Found 64149 images belonging to 16 classes.
Found 14299 images belonging to 16 classes.
Epoch 1/20
2004/2004 [==============================] - 342s 171ms/step - loss: 0.3925 - acc: 0.8682 - val_loss: 0.2238 - val_acc: 0.9278
Epoch 2/20
2004/2004 [=========================

2004/2004 [==============================] - 342s 171ms/step - loss: 0.2026 - acc: 0.9261 - val_loss: 0.2159 - val_acc: 0.9294
Epoch 2/20
2004/2004 [==============================] - 342s 171ms/step - loss: 0.1726 - acc: 0.9356 - val_loss: 0.1867 - val_acc: 0.9364
Epoch 3/20
2004/2004 [==============================] - 342s 171ms/step - loss: 0.1743 - acc: 0.9346 - val_loss: 0.3428 - val_acc: 0.8875
Epoch 4/20
2004/2004 [==============================] - 342s 171ms/step - loss: 0.1734 - acc: 0.9350 - val_loss: 0.2791 - val_acc: 0.9103
Traing  4  done
Saved model to disk

Training  5
Found 65987 images belonging to 16 classes.
Found 12461 images belonging to 16 classes.
Epoch 1/20
2062/2062 [==============================] - 349s 169ms/step - loss: 0.1850 - acc: 0.9305 - val_loss: 0.1917 - val_acc: 0.9358
Epoch 2/20
2062/2062 [==============================] - 349s 169ms/step - loss: 0.1698 - acc: 0.9359 - val_loss: 0.1147 - val_acc: 0.9623
Epoch 3/20
2062/2062 [========================

2004/2004 [==============================] - 342s 171ms/step - loss: 0.1308 - acc: 0.9466 - val_loss: 0.0686 - val_acc: 0.9779
Epoch 3/20
2004/2004 [==============================] - 342s 171ms/step - loss: 0.1330 - acc: 0.9454 - val_loss: 0.1135 - val_acc: 0.9619
Traing  4  done
Saved model to disk

Training  5
Found 65987 images belonging to 16 classes.
Found 12461 images belonging to 16 classes.
Epoch 1/20
2062/2062 [==============================] - 349s 169ms/step - loss: 0.1339 - acc: 0.9459 - val_loss: 0.0475 - val_acc: 0.9842
Epoch 2/20
2062/2062 [==============================] - 349s 169ms/step - loss: 0.1306 - acc: 0.9464 - val_loss: 0.0500 - val_acc: 0.9802
Epoch 3/20
2062/2062 [==============================] - 349s 169ms/step - loss: 0.1228 - acc: 0.9487 - val_loss: 0.0586 - val_acc: 0.9814
Traing  5  done
Saved model to disk

Training  6
Found 68453 images belonging to 16 classes.
Found 9995 images belonging to 16 classes.
Epoch 1/20
2139/2139 [==========================

In [5]:
#테스트셋 불러오기
test_datagen = ImageDataGenerator(horizontal_flip = True)
test_set = validation_datagen.flow_from_directory('./Dataset_Multi/Testset',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(
            test_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")
output = model.predict_generator(test_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_set.class_indices)
print(output[0])

Found 9588 images belonging to 16 classes.
-- Evaluate --
acc: 19.47%
-- Predict --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.051 0.054 0.000 0.001 0.000 0.000 0.000 0.863 0.002 0.000 0.000 0.001
 0.012 0.000 0.015 0.000]


In [6]:
# 모델 평가하기 - 트레이닝 셋으로
print("-- Evaluate --")
scores = model.evaluate_generator(
            training_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기 - 트레이닝 셋으로
print("-- Predict --")
output = model.predict_generator(training_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(training_set.class_indices)
print(output[0])

-- Evaluate --
acc: 94.81%
-- Predict --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.006 0.003 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.001 0.002 0.000 0.988]
